In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import datetime
import seaborn as sns

#import pydicom
import time
from functools import partial
import gc
import operator 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
from sklearn.model_selection import KFold
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from skimage.io import imread,imshow
from helper import *
import helper
import torchvision.models as models
from torch.optim import Adam
from defenitions import *

/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [3]:
SEED = 432
device=device_by_name("Tesla")
#device=device_by_name("RTX")
#device=device_by_name("1060")
torch.cuda.set_device(device)
#device = "cpu"
sendmeemail=Email_Progress(my_gmail,my_pass,to_email,'Densenet161-Copy2-2 results')

In [4]:
def get_submission(test_df,pred):
    epidural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_epidural','Label':torch.sigmoid(pred[:,0])})
    intraparenchymal_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraparenchymal','Label':torch.sigmoid(pred[:,1])})
    intraventricular_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraventricular','Label':torch.sigmoid(pred[:,2])})
    subarachnoid_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subarachnoid','Label':torch.sigmoid(pred[:,3])})
    subdural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subdural','Label':torch.sigmoid(pred[:,4])})
    any_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_any','Label':torch.sigmoid(pred[:,5])}) 
    return pd.concat([epidural_df,
                        intraparenchymal_df,
                        intraventricular_df,
                        subarachnoid_df,
                        subdural_df,
                        any_df]).sort_values('ID').reset_index(drop=True)

In [5]:
def get_submission_ids(image_ids,pred,do_sigmoid=True):
    if do_sigmoid:
        func = lambda x:torch.sigmoid(x)
    else:
        func = lambda x:x
    epidural_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_epidural','Label':func(pred[:,0])})
    intraparenchymal_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_intraparenchymal','Label':func(pred[:,1])})
    intraventricular_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_intraventricular','Label':func(pred[:,2])})
    subarachnoid_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_subarachnoid','Label':func(pred[:,3])})
    subdural_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_subdural','Label':func(pred[:,4])})
    any_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_any','Label':func(pred[:,5])}) 
    return pd.concat([epidural_df,
                        intraparenchymal_df,
                        intraventricular_df,
                        subarachnoid_df,
                        subdural_df,
                        any_df]).sort_values('ID').reset_index(drop=True)

In [6]:
train_df = pd.read_csv(data_dir+'train.csv')
train_df.shape
train_df=train_df[~train_df.PatientID.isin(bad_images)].reset_index(drop=True)
train_df=train_df.drop_duplicates().reset_index(drop=True)
train_df.shape
train_df.head()

(674510, 15)

(674252, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,PID,StudyI,SeriesI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,63eb1e259,0,0,0,0,0,0,a449357f,62d125e5b2,0be5c0d1b3,"['00036', '00036']","['00080', '00080']",180.199951,-125.0,-8.000000
1,2669954a7,0,0,0,0,0,0,363d5865,a20b80c7bf,3564d584db,"['00047', '00047']","['00080', '00080']",922.530821,-156.0,45.572849
2,52c9913b1,0,0,0,0,0,0,9c2b4bd7,3e3634f8cf,973274ffc9,40,150,4.455000,-125.0,-115.063000
3,4e6ff6126,0,0,0,0,0,0,3ae81c2d,a1390c15c2,e5ccad8244,"['00036', '00036']","['00080', '00080']",100.000000,-99.5,28.500000
4,7858edd88,0,0,0,0,0,0,c1867feb,c73e81ed3a,28e0531b3a,40,100,145.793000,-125.0,-132.190000


In [7]:
test_df = pd.read_csv(data_dir+'test.csv')
test_df.head()

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,SeriesI,PID,StudyI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,28fbab7eb,0.5,0.5,0.5,0.5,0.5,0.5,ebfd7e4506,cf1b6b11,93407cadbb,30,80,158.458000,-125.0,-135.598000
1,877923b8b,0.5,0.5,0.5,0.5,0.5,0.5,6d95084e15,ad8ea58f,a337baa067,30,80,138.729050,-125.0,-101.797981
2,a591477cb,0.5,0.5,0.5,0.5,0.5,0.5,8e06b2c9e0,ecfb278b,0cfe838d54,30,80,60.830002,-125.0,-133.300003
3,42217c898,0.5,0.5,0.5,0.5,0.5,0.5,e800f419cf,e96e31f4,c497ac5bad,30,80,55.388000,-125.0,-146.081000
4,a130c4d2f,0.5,0.5,0.5,0.5,0.5,0.5,faeb7454f3,69affa42,854e4fbc01,30,80,33.516888,-125.0,-118.689819


In [8]:
class PosSeriesSampler():
    def __init__(self,df,split,pos_r,neg_r):
        self.df=df[df.SeriesI.isin(split)]
        self.pos_r=max(min(pos_r,1.0),0)
        self.neg_r=max(min(neg_r,1.0),0)
        cols=['SeriesI','any']
        self.series_df=self.df[cols].groupby('SeriesI').sum().reset_index()
        self.series_df['any']=self.series_df['any']>0
        self.sids=self.series_df.SeriesI.values
        self.any_col=self.series_df['any'].values
        self.any_col=self.align(self.any_col,split,self.sids)
        self.pos=np.where(self.any_col)[0]
        self.neg=np.where(~self.any_col)[0]

    def align(self,arr,index1,index2):
        return arr[np.argsort(index2)[np.argsort(np.argsort(index1))]]
    def __call__(self):
        tmpp=self.pos.copy()
        np.random.shuffle(tmpp)
        tmpn=self.neg.copy()
        np.random.shuffle(tmpn)
        return np.concatenate([tmpp[:int(self.pos_r*tmpp.shape[0])],tmpn[:int(self.neg_r*tmpn.shape[0])]])
        


In [9]:
split_sid = train_df.PID.unique()
splits=list(KFold(n_splits=5,shuffle=True, random_state=SEED).split(split_sid))


In [10]:
def my_loss(y_pred,y_true,weights):
    window=(y_true>=0).to(torch.float)
    loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*weights.expand_as(y_true)).mean()/(window.mean()+1e-7)
    return loss

In [11]:
class Metric():
    def __init__(self,weights,k=0.03):
        self.weights=weights
        self.k=k
        self.zero()
        
    def zero(self):
        self.loss_sum=0.
        self.loss_count=0.
        self.lossf=0.
        
    def calc(self,y_pred,y_true,prefix=""):
        window=(y_true>=0).to(torch.float)
        loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*self.weights.expand_as(y_true)).mean()/(window.mean()+1e-5)
        self.lossf=self.lossf*(1-self.k)+loss*self.k
        self.loss_sum=self.loss_sum+loss*window.sum()
        self.loss_count=self.loss_count+window.sum()
        return({prefix+'mloss':self.lossf})    
        
    def calc_sums(self,prefix=""):
        return({prefix+'mloss_tot':self.loss_sum/self.loss_count})    



In [12]:
#features=(features-features.mean())/features.std()

In [13]:
class SimpleModel(nn.Module):
    def __init__(self,in_size):
        super(SimpleModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (7,in_size), padding=(3,0))
        self.bn0=torch.nn.BatchNorm1d(128)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 128, 5, padding=2)
        self.bn1=torch.nn.BatchNorm1d(128)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 3, padding=1)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(64, 6, 3, padding=1)
        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.conv1d1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [14]:
class SimpleModel2(nn.Module):
    def __init__(self,in_size):
        super(SimpleModel2, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(128)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 128, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(128)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(64, 6, 3, padding=1)
        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.conv1d1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [15]:
class ClassModel(nn.Module):
    def __init__(self,in_size):
        super(ClassModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(128)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 128, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(128)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(128, 6, 3, padding=1)
        
        self.conv2d1class=torch.nn.Conv2d(1, 128, (9,in_size), padding=(4,0))
        self.bn0class=torch.nn.BatchNorm1d(128)
        self.maxpool1class=torch.nn.MaxPool1d(3)
        self.conv1d1class=torch.nn.Conv1d(128, 128, 3, padding=1)
        self.bn1class=torch.nn.BatchNorm1d(128)
        self.maxpool2class=torch.nn.MaxPool1d(3)
        self.conv1d2class=torch.nn.Conv1d(128, 64, 2, padding=1)
        self.bn2class=torch.nn.BatchNorm1d(64)

        
        
    def forward(self, x):
        z=x
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.conv1d1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        z=self.conv2d1class(z.unsqueeze(1)).squeeze(-1)
        z=self.bn0class(z)
        z=self.maxpool1class(z)
        z=self.conv1d1class(z)
        z=self.maxpool2class(z)
        z=self.conv1d2class(z)
        z=self.bn2class(z)
        z=F.max_pool1d(z,kernel_size=z.shape[-1])
        z=z.expand_as(x)
        x=torch.cat([x,z],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [16]:
class ResModel(nn.Module):
    def __init__(self,in_size):
        super(ResModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        
        
    def forward(self, x):
        x=x.unsqueeze(1)
        x = self.conv2d1(x).squeeze(-1)
        x = self.bn0(x)
        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [17]:
class ResModelPool(nn.Module):
    def __init__(self,in_size):
        super(ResModelPool, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size),stride=(1,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
#        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        
        
    def forward(self, x):
        x=x.unsqueeze(1)
        x = self.conv2d1(x)
        x=F.max_pool2d(x,kernel_size=(1,x.shape[-1])).squeeze(-1)        
        x0 = self.bn0(x)
#        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [18]:
class ResDropModel(nn.Module):
    def __init__(self,in_size,dropout=0.2):
        super(ResDropModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        self.dropout=dropout
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = F.dropout(x,self.dropout)
        x0 = self.relu0(x) 
        x = self.conv1d1(x0)
#        x = self.bn1(x)
        x = F.dropout(x,self.dropout)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
#        x = self.bn2(x)
        x = F.dropout(x,self.dropout)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        x = F.dropout(x,self.dropout)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [19]:
fn = partial(torch.clamp,min=0,max=1)

In [20]:
class GenReLU(nn.Module):
    def __init__(self,leak=0,add=0,clamp=None):
        super(GenReLU, self).__init__()
        self.leak,self.add=leak,add
        if isinstance(clamp,tuple):
            self.clamp = partial(torch.clamp,min=clamp[0],max=clamp[1])
        elif clamp:
            self.clamp = partial(torch.clamp,min=-clamp,max=clamp)
        else:
            self.clamp=None
            
        
    def forward(self,x):
        x = F.leaky_relu(x,self.leak)
        if self.add:
            x=x+self.add
        if self.clamp:
            x = self.clamp(x)
        return x
        
        

In [21]:

class ResModelIn(nn.Module):
    def __init__(self,in_size):
        super(ResModelIn, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 3, padding=1)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 3, padding=1)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)

        
        
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv2d1(x).squeeze(-1)
        x = self.bn0(x)
        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        x = self.conv1d3(x)
        out = x.transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [22]:

class ResModelInR(nn.Module):
    def __init__(self,in_size):
        super(ResModelInR, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0= GenReLU(leak=0.01,add=-0.3,clamp=(-1,3))
        self.conv1d1=torch.nn.Conv1d(64, 64, 3, padding=1)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=GenReLU(leak=0.01,add=-0.3,clamp=(-1,3))
        self.conv1d2=torch.nn.Conv1d(128, 64, 3, padding=1)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=GenReLU(leak=0,add=0,clamp=(0,2))
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)

        
        
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv2d1(x).squeeze(-1)
        x = self.bn0(x)
        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        x = self.conv1d3(x)
        out = x.transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [23]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (5,2208), padding=(2,0))
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 6, 1)

        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.relu0(x)
        out = self.conv1d1(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [23]:
%matplotlib nbagg
for num_split in [0]: #range(5):
    multi=3
    model_name,version = 'se_resnext101_32x4d' , 'new_splits_focal'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape

    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    split_train = train_df[train_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=18
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(train_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(train_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_over{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
#    sampling=PosSeriesSampler(train_df,split_train,1,0.75)
    sample_ratio= 1.0 #1.01*float(sampling().shape[0])/split_train.shape[0]
    num_train_optimization_steps = sample_ratio*num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnext101_32x4d new_splits_focal 0


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

{'loss': 0.067308146525303, 'mloss': tensor(0.0673), 'val_loss': 0.07460971067906642, 'val_mloss_tot': tensor(0.0745)}


{'loss': 0.06447429880488798, 'mloss': tensor(0.0628), 'val_loss': 0.07257415679644565, 'val_mloss_tot': tensor(0.0725)}


...

{'loss': 0.05543938604505698, 'mloss': tensor(0.0556), 'val_loss': 0.068675661115546, 'val_mloss_tot': tensor(0.0686)}


{'loss': 0.055983229030143716, 'mloss': tensor(0.0528), 'val_loss': 0.06827947101460732, 'val_mloss_tot': tensor(0.0682)}


{'loss': 0.055272456923741504, 'mloss': tensor(0.0555), 'val_loss': 0.06822937811363716, 'val_mloss_tot': tensor(0.0682)}


{'loss': 0.057854696961629044, 'mloss': tensor(0.0576), 'val_loss': 0.06831360294730687, 'val_mloss_tot': tensor(0.0683)}




(0.06820877346837399, {'val_loss': 0.06820877346837399, 'val_mloss_tot': tensor(0.0682)})


In [24]:
%matplotlib nbagg
for num_split in [1]: #range(5):
    multi=3
    model_name,version = 'se_resnext101_32x4d' , 'new_splits_focal'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape

    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    split_train = train_df[train_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=18
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(train_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(train_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_over{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
#    sampling=PosSeriesSampler(train_df,split_train,1,0.75)
    sample_ratio= 1.0 #1.01*float(sampling().shape[0])/split_train.shape[0]
    num_train_optimization_steps = sample_ratio*num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnext101_32x4d new_splits_focal 1


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

{'loss': 0.06608332269690445, 'mloss': tensor(0.0675), 'val_loss': 0.07562760766595603, 'val_mloss_tot': tensor(0.0755)}


{'loss': 0.06423676105824865, 'mloss': tensor(0.0646), 'val_loss': 0.07186336961652463, 'val_mloss_tot': tensor(0.0718)}


...

{'loss': 0.05712647444272771, 'mloss': tensor(0.0580), 'val_loss': 0.06951607308195283, 'val_mloss_tot': tensor(0.0694)}


{'loss': 0.06183372723698266, 'mloss': tensor(0.0638), 'val_loss': 0.06949221545364707, 'val_mloss_tot': tensor(0.0694)}


{'loss': 0.058854192375465825, 'mloss': tensor(0.0561), 'val_loss': 0.06922740353426586, 'val_mloss_tot': tensor(0.0691)}


{'loss': 0.058280917904031275, 'mloss': tensor(0.0580), 'val_loss': 0.06973961847058187, 'val_mloss_tot': tensor(0.0696)}




0.06914957696959997


In [26]:
%matplotlib nbagg
for num_split in [2,3]: #range(5):
    multi=3
    model_name,version = 'se_resnext101_32x4d' , 'new_splits_focal'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape

    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    split_train = train_df[train_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=18
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(train_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(train_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_over{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
#    sampling=PosSeriesSampler(train_df,split_train,1,0.75)
    sample_ratio= 1.0 #1.01*float(sampling().shape[0])/split_train.shape[0]
    num_train_optimization_steps = sample_ratio*num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnext101_32x4d new_splits_focal 2


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

{'loss': 0.06811236130900705, 'mloss': tensor(0.0645), 'val_loss': 0.07296392299717495, 'val_mloss_tot': tensor(0.0729)}


{'loss': 0.06484684772493185, 'mloss': tensor(0.0617), 'val_loss': 0.07182952273365578, 'val_mloss_tot': tensor(0.0718)}


...

{'loss': 0.054036270286534786, 'mloss': tensor(0.0531), 'val_loss': 0.07390088248486462, 'val_mloss_tot': tensor(0.0738)}


{'loss': 0.054036270286534786, 'mloss': tensor(0.0531), 'val_loss': 0.07390088248486462, 'val_mloss_tot': tensor(0.0738)}


{'loss': 0.054714143508980885, 'mloss': tensor(0.0551), 'val_loss': 0.07393516586009474, 'val_mloss_tot': tensor(0.0739)}



{'loss': 0.054714143508980885, 'mloss': tensor(0.0551), 'val_loss': 0.07393516586009474, 'val_mloss_tot': tensor(0.0739)}




0.07320393128778602

0.07320393128778602


In [27]:
%matplotlib nbagg
for num_split in [4]: #range(5):
    multi=3
    model_name,version = 'se_resnext101_32x4d' , 'new_splits_focal'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape

    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    split_train = train_df[train_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=18
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(train_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(train_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_over{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
#    sampling=PosSeriesSampler(train_df,split_train,1,0.75)
    sample_ratio= 1.0 #1.01*float(sampling().shape[0])/split_train.shape[0]
    num_train_optimization_steps = sample_ratio*num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnext101_32x4d new_splits_focal 4
se_resnext101_32x4d new_splits_focal 4


torch.Size([2697008, 256])

torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

torch.Size([674252, 4, 256])

...

{'loss': 0.05750886167725116, 'mloss': tensor(0.0591), 'val_loss': 0.07257047332521634, 'val_mloss_tot': tensor(0.0725)}


{'loss': 0.05750886167725116, 'mloss': tensor(0.0591), 'val_loss': 0.07257047332521634, 'val_mloss_tot': tensor(0.0725)}


{'loss': 0.05547570323233913, 'mloss': tensor(0.0540), 'val_loss': 0.0725123643053145, 'val_mloss_tot': tensor(0.0725)}



{'loss': 0.05547570323233913, 'mloss': tensor(0.0540), 'val_loss': 0.0725123643053145, 'val_mloss_tot': tensor(0.0725)}




(0.07146647619649105, {'val_loss': 0.07146647619649105, 'val_mloss_tot': tensor(0.0714)})

(0.07146647619649105, {'val_loss': 0.07146647619649105, 'val_mloss_tot': tensor(0.0714)})


In [24]:
%matplotlib nbagg
for num_split in [0]: #range(5):
    multi=3
    model_name,version = 'se_resnet101' , 'new_splits_focal'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape

    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    split_train = train_df[train_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=18
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(train_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(train_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_over{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
#    sampling=PosSeriesSampler(train_df,split_train,1,0.75)
    sample_ratio= 1.0 #1.01*float(sampling().shape[0])/split_train.shape[0]
    num_train_optimization_steps = sample_ratio*num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnet101 new_splits_focal 0


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

{'loss': 0.06801814952788739, 'mloss': tensor(0.0687), 'val_loss': 0.07283909177901793, 'val_mloss_tot': tensor(0.0728)}


{'loss': 0.06713587075366854, 'mloss': tensor(0.0654), 'val_loss': 0.07061037322012137, 'val_mloss_tot': tensor(0.0705)}


...

{'loss': 0.05948680389709346, 'mloss': tensor(0.0594), 'val_loss': 0.06839205849611638, 'val_mloss_tot': tensor(0.0683)}


{'loss': 0.06009998340068295, 'mloss': tensor(0.0568), 'val_loss': 0.06836220631185844, 'val_mloss_tot': tensor(0.0683)}


{'loss': 0.058883878685634106, 'mloss': tensor(0.0587), 'val_loss': 0.06831089384207616, 'val_mloss_tot': tensor(0.0682)}


{'loss': 0.06160811806986899, 'mloss': tensor(0.0609), 'val_loss': 0.06832941194844185, 'val_mloss_tot': tensor(0.0683)}


0.06814609903043935


In [25]:
%matplotlib nbagg
for num_split in [1]: #range(5):
    multi=3
    model_name,version = 'se_resnet101' , 'new_splits_focal'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape

    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    split_train = train_df[train_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=18
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(train_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(train_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_over{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
#    sampling=PosSeriesSampler(train_df,split_train,1,0.75)
    sample_ratio= 1.0 #1.01*float(sampling().shape[0])/split_train.shape[0]
    num_train_optimization_steps = sample_ratio*num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnet101 new_splits_focal 1


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

{'loss': 0.06706305390403391, 'mloss': tensor(0.0679), 'val_loss': 0.0753338036360219, 'val_mloss_tot': tensor(0.0752)}


{'loss': 0.06582605521661761, 'mloss': tensor(0.0664), 'val_loss': 0.07156634539132938, 'val_mloss_tot': tensor(0.0715)}


...

{'loss': 0.05841321709674182, 'mloss': tensor(0.0592), 'val_loss': 0.06885637667728588, 'val_mloss_tot': tensor(0.0688)}


{'loss': 0.06359648862342149, 'mloss': tensor(0.0650), 'val_loss': 0.06877168237309282, 'val_mloss_tot': tensor(0.0687)}


{'loss': 0.060309677044462016, 'mloss': tensor(0.0571), 'val_loss': 0.06850194214687993, 'val_mloss_tot': tensor(0.0684)}


{'loss': 0.06016276608588028, 'mloss': tensor(0.0603), 'val_loss': 0.06883820214619239, 'val_mloss_tot': tensor(0.0687)}


0.06850194214687993


In [26]:
%matplotlib nbagg
for num_split in [2]: #range(5):
    multi=3
    model_name,version = 'se_resnet101' , 'new_splits_focal'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape

    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    split_train = train_df[train_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=18
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(train_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(train_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_over{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
#    sampling=PosSeriesSampler(train_df,split_train,1,0.75)
    sample_ratio= 1.0 #1.01*float(sampling().shape[0])/split_train.shape[0]
    num_train_optimization_steps = sample_ratio*num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnet101 new_splits_focal 2


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

{'loss': 0.06642876009938492, 'mloss': tensor(0.0636), 'val_loss': 0.072511745141706, 'val_mloss_tot': tensor(0.0725)}


{'loss': 0.06366300585780155, 'mloss': tensor(0.0610), 'val_loss': 0.07134100720889655, 'val_mloss_tot': tensor(0.0713)}


...

{'loss': 0.0608769425321196, 'mloss': tensor(0.0617), 'val_loss': 0.06982114527992873, 'val_mloss_tot': tensor(0.0698)}


{'loss': 0.05789883858367275, 'mloss': tensor(0.0598), 'val_loss': 0.06970915781628144, 'val_mloss_tot': tensor(0.0697)}


{'loss': 0.06090872075104362, 'mloss': tensor(0.0627), 'val_loss': 0.06969711087598854, 'val_mloss_tot': tensor(0.0697)}


{'loss': 0.059125143559363255, 'mloss': tensor(0.0586), 'val_loss': 0.06957406156624739, 'val_mloss_tot': tensor(0.0695)}


0.06941451203937714


In [27]:
%matplotlib nbagg
for num_split in [3]: #range(5):
    multi=3
    model_name,version = 'se_resnet101' , 'new_splits_focal'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape

    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    split_train = train_df[train_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=18
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(train_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(train_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_over{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
#    sampling=PosSeriesSampler(train_df,split_train,1,0.75)
    sample_ratio= 1.0 #1.01*float(sampling().shape[0])/split_train.shape[0]
    num_train_optimization_steps = sample_ratio*num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnet101 new_splits_focal 3


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

{'loss': 0.06795507064264683, 'mloss': tensor(0.0664), 'val_loss': 0.07484717968255893, 'val_mloss_tot': tensor(0.0748)}


{'loss': 0.06469026769797935, 'mloss': tensor(0.0645), 'val_loss': 0.07318627575916223, 'val_mloss_tot': tensor(0.0731)}


...

{'loss': 0.06110199735701955, 'mloss': tensor(0.0621), 'val_loss': 0.07118308365146522, 'val_mloss_tot': tensor(0.0711)}


{'loss': 0.059016008194910985, 'mloss': tensor(0.0607), 'val_loss': 0.07190553739392122, 'val_mloss_tot': tensor(0.0718)}


{'loss': 0.05790989192603154, 'mloss': tensor(0.0569), 'val_loss': 0.07156969215835399, 'val_mloss_tot': tensor(0.0715)}


{'loss': 0.05835562291724042, 'mloss': tensor(0.0585), 'val_loss': 0.07158829269884917, 'val_mloss_tot': tensor(0.0715)}


0.07118308365146522


In [44]:
%matplotlib nbagg
for num_split in [4]: #range(5):
    multi=3
    model_name,version = 'se_resnet101' , 'new_splits_focal'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape

    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    split_train = train_df[train_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=18
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(train_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(train_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_over{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
#    sampling=PosSeriesSampler(train_df,split_train,1,0.75)
    sample_ratio= 1.0 #1.01*float(sampling().shape[0])/split_train.shape[0]
    num_train_optimization_steps = sample_ratio*num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnet101 new_splits_focal 4


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

{'loss': 0.07015490436772923, 'mloss': tensor(0.0726), 'val_loss': 0.07512244625160328, 'val_mloss_tot': tensor(0.0751)}


{'loss': 0.0653958427481258, 'mloss': tensor(0.0676), 'val_loss': 0.07276976575343473, 'val_mloss_tot': tensor(0.0727)}


...

{'loss': 0.05662640915423674, 'mloss': tensor(0.0543), 'val_loss': 0.07070589078748636, 'val_mloss_tot': tensor(0.0706)}


{'loss': 0.05799873370762432, 'mloss': tensor(0.0545), 'val_loss': 0.07046750419384797, 'val_mloss_tot': tensor(0.0704)}


{'loss': 0.05963707518961883, 'mloss': tensor(0.0607), 'val_loss': 0.07042344067215557, 'val_mloss_tot': tensor(0.0704)}


{'loss': 0.05862137302747463, 'mloss': tensor(0.0576), 'val_loss': 0.07037128592173943, 'val_mloss_tot': tensor(0.0703)}


(0.06990519024220555, {'val_loss': 0.06990519024220555, 'val_mloss_tot': tensor(0.0698)})


In [ ]:
multi=3
model_names=['se_resnext101_32x4d']
types=['features_train_tta']
versions=['new_splits_focal']
num_splits=[5]
seeds=[432]
for model_name,type_,version_,n,SEED in zip(model_names,types,versions,num_splits,seeds):
    for num_split in tqdm_notebook(range(n)):
        split_sid = train_df.PID.unique()
        splits=list(KFold(n_splits=n,shuffle=True, random_state=SEED).split(split_sid))

        pred_list=[]
        print(model_name,version_,type_,num_split) 
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        features=pickle.load(pickle_file)
        pickle_file.close()
        features=features.reshape(features.shape[0]//4,4,-1)
        split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()
        model=ResModelPool(features.shape[-1])
        version=version_+'_fullhead_resmodel_pool2_over{}'.format(multi)

        model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))

        valid_dataset=FullHeadDataset(train_df,
                                      split_validate,
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                      multi =3)

        win_dataset=FullHeadDataset(train_df,
                                      split_validate,
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                       target_columns=hemorrhage_types)
        win_list=[]
        dl = D.DataLoader(win_dataset,batch_size=128,num_workers=16)
        for _,win in tqdm_notebook(dl):
            win_list.append(win.reshape(win.shape[0]*win.shape[1],-1))    
        wins = torch.cat(win_list,0).sum(1)>=0
        wins.sum()
        for i in tqdm_notebook(range(32),leave=False):
            pr = model_run(model,valid_dataset,do_apex=False,batch_size=128)
            pred_list.append(pr.reshape(pr.shape[0]*pr.shape[1],-1)[wins])
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'OOF_pred',num_split),'wb')
        pickle.dump(pred_list,pickle_file,protocol=4)
        pickle_file.close()


se_resnext101_32x4d new_splits_focal features_train_tta 0


<All keys matched successfully>

tensor(134734)

In [47]:
multi=3
model_names=['se_resnet101']
types=['features_train_tta']
versions=['new_splits_focal']
num_splits=[5]
seeds=[432]
for model_name,type_,version_,n,SEED in zip(model_names,types,versions,num_splits,seeds):
    for num_split in tqdm_notebook(range(n)):
        split_sid = train_df.PID.unique()
        splits=list(KFold(n_splits=n,shuffle=True, random_state=SEED).split(split_sid))

        pred_list=[]
        print(model_name,version_,type_,num_split) 
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        features=pickle.load(pickle_file)
        pickle_file.close()
        features=features.reshape(features.shape[0]//4,4,-1)
        split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()
        model=ResModelPool(features.shape[-1])
        version=version_+'_fullhead_resmodel_pool2_over{}'.format(multi)

        model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))

        valid_dataset=FullHeadDataset(train_df,
                                      split_validate,
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                      multi =3)

        win_dataset=FullHeadDataset(train_df,
                                      split_validate,
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                       target_columns=hemorrhage_types)
        win_list=[]
        dl = D.DataLoader(win_dataset,batch_size=128,num_workers=16)
        for _,win in tqdm_notebook(dl):
            win_list.append(win.reshape(win.shape[0]*win.shape[1],-1))    
        wins = torch.cat(win_list,0).sum(1)>=0
        wins.sum()
        for i in tqdm_notebook(range(32),leave=False):
            pr = model_run(model,valid_dataset,do_apex=False,batch_size=128)
            pred_list.append(pr.reshape(pr.shape[0]*pr.shape[1],-1)[wins])
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'OOF_pred',num_split),'wb')
        pickle.dump(pred_list,pickle_file,protocol=4)
        pickle_file.close()


se_resnet101 new_splits_focal features_train_tta 0


<All keys matched successfully>

tensor(134734)

se_resnet101 new_splits_focal features_train_tta 1


<All keys matched successfully>

...

tensor(135582)

se_resnet101 new_splits_focal features_train_tta 4


<All keys matched successfully>

tensor(136008)

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/reina/anaconda3/envs/RSNA/lib/python3.6/multiprocessing/queues.py", line 230, in _feed
    close()
  File "/home/reina/anaconda3/envs/RSNA/lib/python3.6/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/reina/anaconda3/envs/RSNA/lib/python3.6/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/home/reina/anaconda3/envs/RSNA/lib/python3.6/multiprocessing/queues.py", line 230, in _feed
    close()
  File "/home/reina/anaconda3/envs/RSNA/lib/python3.6/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/reina/anaconda3/envs/RSNA/lib/python3.6/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
  File "/home/reina/anaconda3/envs/RSNA/lib/python3.6/multiprocessing/queues.py", l

In [32]:
pickle_file=open(outputs_dir+'OOF_validation_image_ids_5.pkl','rb')
image_id_folds=pickle.load(pickle_file)
pickle_file.close()


In [91]:

targets=res_df.loc[image_id_folds[0]]

In [92]:
targets.shape

(134734, 6)

In [102]:
multi=3
cols=['PatientID']
cols.extend(hemorrhage_types)
res_df=train_df[cols].set_index('PatientID')
model_names=['se_resnet101']
types=['OOF_pred']
versions=['new_splits_focal_fullhead_resmodel_pool2_over3']
num_splits=[5]
#model_names=['se_resnext101_32x4d','se_resnet101']
#types=['test_pred_ensamble','test_pred_ensamble']
#versions=['new_splits_fullhead_resmodel_pool2_3','new_splits_fullhead_resmodel_pool2_3']
#num_splits=[5,5]
#model_names=['se_resnet101']
#types=['OOF_pred']
#versions=['new_splits_focal_fullhead_resmodel_pool2_over3']
#num_splits=[5]
l=0.
for num_split in range(n):
    pred_list=[]
    for model_name,type_,version_,n in zip(model_names,types,versions,num_splits):
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        pred_list.extend(pickle.load(pickle_file))
        pickle_file.close()
    preds=torch.cat([p[None] for p in pred_list],0).mean(0)
    targets=torch.tensor(res_df.loc[image_id_folds[num_split]][hemorrhage_types].values,dtype=torch.float)
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],dtype=torch.float)
    my_loss(preds,targets,weights)
    l=l+my_loss(preds,targets,weights)
l/5


tensor(0.0680)

tensor(0.0683)

tensor(0.0693)

tensor(0.0710)

tensor(0.0698)

tensor(0.0693)

In [103]:
multi=3
cols=['PatientID']
cols.extend(hemorrhage_types)
res_df=train_df[cols].set_index('PatientID')
model_names=['se_resnet101']
types=['OOF_pred']
versions=['new_splits_fullhead_resmodel_pool2_3']
num_splits=[5]
#model_names=['se_resnext101_32x4d','se_resnet101']
#types=['test_pred_ensamble','test_pred_ensamble']
#versions=['new_splits_fullhead_resmodel_pool2_3','new_splits_fullhead_resmodel_pool2_3']
#num_splits=[5,5]
#model_names=['se_resnet101']
#types=['OOF_pred']
#versions=['new_splits_focal_fullhead_resmodel_pool2_over3']
#num_splits=[5]
l=0.
for num_split in range(n):
    pred_list=[]
    for model_name,type_,version_,n in zip(model_names,types,versions,num_splits):
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        pred_list.extend(pickle.load(pickle_file))
        pickle_file.close()
    preds=torch.cat([p[None] for p in pred_list],0).mean(0)
    targets=torch.tensor(res_df.loc[image_id_folds[num_split]][hemorrhage_types].values,dtype=torch.float)
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],dtype=torch.float)
    my_loss(preds,targets,weights)
    l=l+my_loss(preds,targets,weights)
l/5


tensor(0.0675)

tensor(0.0683)

tensor(0.0690)

tensor(0.0716)

tensor(0.0701)

tensor(0.0693)

In [100]:
multi=3
cols=['PatientID']
cols.extend(hemorrhage_types)
res_df=train_df[cols].set_index('PatientID')
model_names=['se_resnext101_32x4d','se_resnet101','se_resnet101']
types=['OOF_pred','OOF_pred','OOF_pred']
versions=['new_splits_fullhead_resmodel_pool2_3','new_splits_fullhead_resmodel_pool2_3','new_splits_focal_fullhead_resmodel_pool2_over3']
num_splits=[5,5,5]
#model_names=['se_resnext101_32x4d','se_resnet101']
#types=['test_pred_ensamble','test_pred_ensamble']
#versions=['new_splits_fullhead_resmodel_pool2_3','new_splits_fullhead_resmodel_pool2_3']
#num_splits=[5,5]
#model_names=['se_resnet101']
#types=['OOF_pred']
#versions=['new_splits_focal_fullhead_resmodel_pool2_over3']
#num_splits=[5]
l=0.
for num_split in range(n):
    pred_list=[]
    for model_name,type_,version_,n in zip(model_names,types,versions,num_splits):
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        pred_list.extend(pickle.load(pickle_file))
        pickle_file.close()
    preds=torch.cat([p[None] for p in pred_list],0).mean(0)
    targets=torch.tensor(res_df.loc[image_id_folds[num_split]][hemorrhage_types].values,dtype=torch.float)
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],dtype=torch.float)
    my_loss(preds,targets,weights)
    l=l+my_loss(preds,targets,weights)
l/5


tensor(0.0664)

tensor(0.0672)

tensor(0.0678)

tensor(0.0701)

tensor(0.0687)

tensor(0.0680)

In [33]:
multi=3
cols=['PatientID']
cols.extend(hemorrhage_types)
res_df=train_df[cols].set_index('PatientID')
model_names=['se_resnext101_32x4d','se_resnext101_32x4d','se_resnet101','se_resnet101']
types=['OOF_pred','OOF_pred','OOF_pred','OOF_pred']
versions=['new_splits_fullhead_resmodel_pool2_3','new_splits_focal_fullhead_resmodel_pool2_over3','new_splits_fullhead_resmodel_pool2_3','new_splits_focal_fullhead_resmodel_pool2_over3']
num_splits=[5,5,5,5]
#model_names=['se_resnext101_32x4d','se_resnet101']
#types=['test_pred_ensamble','test_pred_ensamble']
#versions=['new_splits_fullhead_resmodel_pool2_3','new_splits_fullhead_resmodel_pool2_3']
#num_splits=[5,5]
#model_names=['se_resnet101']
#types=['OOF_pred']
#versions=['new_splits_focal_fullhead_resmodel_pool2_over3']
#num_splits=[5]
l=0.
for num_split in [0,2,3,4]: #range(n):
    pred_list=[]
    for model_name,type_,version_,n in zip(model_names,types,versions,num_splits):
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        pred_list.extend(pickle.load(pickle_file))
        pickle_file.close()
    preds=torch.cat([p[None] for p in pred_list],0).mean(0)
    targets=torch.tensor(res_df.loc[image_id_folds[num_split]][hemorrhage_types].values,dtype=torch.float)
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],dtype=torch.float)
    my_loss(preds,targets,weights)
    l=l+my_loss(preds,targets,weights)
l/4


tensor(0.0662)

tensor(0.0662)

tensor(0.0675)

tensor(0.0675)

tensor(0.0702)

tensor(0.0702)

tensor(0.0687)

tensor(0.0687)

tensor(0.0682)

tensor(0.0682)

In [101]:
multi=3
cols=['PatientID']
cols.extend(hemorrhage_types)
res_df=train_df[cols].set_index('PatientID')
#model_names=['se_resnext101_32x4d','se_resnet101','se_resnet101']
#types=['OOF_pred','OOF_pred','OOF_pred']
#versions=['new_splits_fullhead_resmodel_pool2_3','new_splits_fullhead_resmodel_pool2_3','new_splits_focal_fullhead_resmodel_pool2_over3']
#num_splits=[5,5,5]
model_names=['se_resnext101_32x4d','se_resnet101']
types=['OOF_pred','OOF_pred']
versions=['new_splits_fullhead_resmodel_pool2_3','new_splits_fullhead_resmodel_pool2_3']
num_splits=[5,5]
l=0
#model_names=['se_resnet101']
#types=['OOF_pred']
#versions=['new_splits_focal_fullhead_resmodel_pool2_over3']
#num_splits=[5]
for num_split in range(n):
    pred_list=[]
    for model_name,type_,version_,n in zip(model_names,types,versions,num_splits):
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        pred_list.extend(pickle.load(pickle_file))
        pickle_file.close()
    preds=torch.cat([p[None] for p in pred_list],0).mean(0)
    targets=torch.tensor(res_df.loc[image_id_folds[num_split]][hemorrhage_types].values,dtype=torch.float)
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],dtype=torch.float)
    my_loss(preds,targets,weights)
    l=l+my_loss(preds,targets,weights)
l/5


tensor(0.0664)

tensor(0.0675)

tensor(0.0678)

tensor(0.0705)

tensor(0.0692)

tensor(0.0683)

In [105]:
multi=3
model_names=['se_resnet101']
types=['features_test']
versions=['new_splits_focal']
num_splits=[5]
seeds=[432]
for model_name,type_,version_,n,SEED in zip(model_names,types,versions,num_splits,seeds):
    for num_split in tqdm_notebook(range(n)):
        pred_list=[]
        print(model_name,version_,type_,num_split) 
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        features=pickle.load(pickle_file)
        pickle_file.close()
        features=features.reshape(features.shape[0]//8,8,-1)
        print(features.shape)
        model=ResModelPool(features.shape[-1])
        version=version_+'_fullhead_resmodel_pool2_over{}'.format(multi)

        model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))

        valid_dataset=FullHeadDataset(test_df,
                                      test_df.SeriesI.unique(),
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                      multi =4)

        win_dataset=FullHeadDataset(test_df,
                                      test_df.SeriesI.unique(),
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                       target_columns=hemorrhage_types)
        win_list=[]
        dl = D.DataLoader(win_dataset,batch_size=128,num_workers=16)
        for _,win in tqdm_notebook(dl):
            win_list.append(win.reshape(win.shape[0]*win.shape[1],-1))    
        wins = torch.cat(win_list,0).sum(1)>=0
        wins.sum()
        for i in tqdm_notebook(range(32),leave=False):
            pr = model_run(model,valid_dataset,do_apex=False,batch_size=128)
            pred_list.append(pr.reshape(pr.shape[0]*pr.shape[1],-1)[wins])
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'test_pred_ensamble',num_split),'wb')
        pickle.dump(pred_list,pickle_file,protocol=4)
        pickle_file.close()


se_resnet101 new_splits_focal features_test 0
torch.Size([78545, 8, 256])


<All keys matched successfully>

tensor(78545)

se_resnet101 new_splits_focal features_test 1
torch.Size([78545, 8, 256])


<All keys matched successfully>

...

<All keys matched successfully>

tensor(78545)

se_resnet101 new_splits_focal features_test 4
torch.Size([78545, 8, 256])


<All keys matched successfully>

tensor(78545)

In [34]:
multi=3
model_names=['se_resnext101_32x4d']
types=['features_test']
versions=['new_splits_focal']
num_splits=[5]
seeds=[432]
for model_name,type_,version_,n,SEED in zip(model_names,types,versions,num_splits,seeds):
    for num_split in tqdm_notebook(range(5)):
        pred_list=[]
        print(model_name,version_,type_,num_split) 
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        features=pickle.load(pickle_file)
        pickle_file.close()
        features=features.reshape(features.shape[0]//8,8,-1)
        print(features.shape)
        model=ResModelPool(features.shape[-1])
        version=version_+'_fullhead_resmodel_pool2_over{}'.format(multi)

        model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))

        valid_dataset=FullHeadDataset(test_df,
                                      test_df.SeriesI.unique(),
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                      multi =4)

        win_dataset=FullHeadDataset(test_df,
                                      test_df.SeriesI.unique(),
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                       target_columns=hemorrhage_types)
        win_list=[]
        dl = D.DataLoader(win_dataset,batch_size=128,num_workers=16)
        for _,win in tqdm_notebook(dl):
            win_list.append(win.reshape(win.shape[0]*win.shape[1],-1))    
        wins = torch.cat(win_list,0).sum(1)>=0
        wins.sum()
        for i in tqdm_notebook(range(32),leave=False):
            pr = model_run(model,valid_dataset,do_apex=False,batch_size=128)
            pred_list.append(pr.reshape(pr.shape[0]*pr.shape[1],-1)[wins])
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'test_pred_ensamble',num_split),'wb')
        pickle.dump(pred_list,pickle_file,protocol=4)
        pickle_file.close()


se_resnext101_32x4d new_splits_focal features_test 0
se_resnext101_32x4d new_splits_focal features_test 0
torch.Size([78545, 8, 256])
torch.Size([78545, 8, 256])


<All keys matched successfully>

<All keys matched successfully>

tensor(78545)

tensor(78545)

...

se_resnext101_32x4d new_splits_focal features_test 4
se_resnext101_32x4d new_splits_focal features_test 4
torch.Size([78545, 8, 256])
torch.Size([78545, 8, 256])


<All keys matched successfully>

<All keys matched successfully>

tensor(78545)

tensor(78545)

In [106]:
multi=3
pred_list=[]
model_names=['se_resnext101_32x4d','se_resnet101','se_resnet101']
types=['test_pred_ensamble','test_pred_ensamble','test_pred_ensamble']
versions=['new_splits_fullhead_resmodel_pool2_3','new_splits_fullhead_resmodel_pool2_3','new_splits_focal_fullhead_resmodel_pool2_over3']
num_splits=[5,5,5]
#model_names=['se_resnext101_32x4d','se_resnet101']
#types=['test_pred_ensamble','test_pred_ensamble']
#versions=['new_splits_fullhead_resmodel_pool2_3','new_splits_fullhead_resmodel_pool2_3']
#num_splits=[5,5]
#model_names=['se_resnet101']
#types=['test_pred_ensamble']
#versions=['new_splits_focal_fullhead_resmodel_pool2_over3']
#num_splits=[5]

for model_name,type_,version_,n in zip(model_names,types,versions,num_splits):
    for num_split in range(n):
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        pred_list.extend(pickle.load(pickle_file))
        pickle_file.close()


In [107]:
preds=torch.sigmoid(torch.cat([p[None] for p in pred_list],0)).mean(0)

In [108]:
preds.shape
preds[:10]

torch.Size([78545, 6])

tensor([[3.6248e-05, 2.9047e-05, 6.5163e-06, 7.4847e-05, 7.3615e-05, 1.5056e-04],
        [4.0630e-06, 3.1331e-06, 3.9070e-07, 1.3053e-05, 1.4722e-05, 4.0180e-05],
        [3.4322e-05, 8.7259e-05, 2.1853e-06, 1.7809e-04, 1.1406e-04, 4.1449e-04],
        [8.9950e-05, 2.7716e-04, 5.7556e-06, 2.6430e-04, 3.0350e-04, 8.5183e-04],
        [7.7688e-05, 2.5398e-04, 9.0096e-06, 2.0069e-04, 2.5038e-04, 6.7380e-04],
        [5.7845e-05, 1.6590e-04, 7.1271e-06, 1.6075e-04, 2.6374e-04, 5.0952e-04],
        [4.1528e-05, 1.3285e-04, 6.4271e-06, 1.3693e-04, 3.2159e-04, 5.3201e-04],
        [3.6813e-05, 1.4233e-04, 7.8734e-06, 1.5962e-04, 4.4771e-04, 7.4803e-04],
        [2.9815e-05, 1.1054e-04, 5.6925e-06, 1.3346e-04, 4.1353e-04, 6.1622e-04],
        [3.0067e-05, 1.5906e-04, 7.3135e-06, 1.6750e-04, 4.4270e-04, 8.1449e-04]])

In [109]:
pickle_file=open(outputs_dir+'ensemble_test_image_ids.pkl','rb')
image_ids=pickle.load(pickle_file)
pickle_file.close()


In [110]:
image_ids[:10]

array(['37874f3c6', '2e16b8f05', '0b758b9c0', 'b8eba0e41', 'acf5e1ac4',
       'ae3c642b0', '7edd07ece', '2cbe4682d', '7ed61df92', 'df94f381c'],
      dtype=object)

In [111]:
submission_df=get_submission_ids(image_ids,preds,do_sigmoid=False)
submission_df.head(12)
submission_df.shape
#sub_num=57
#submission_df.to_csv('/media/hd/notebooks/data/RSNA/submissions/submission{}.csv'.format(sub_num),
#                                                                  index=False, columns=['ID','Label'])


,ID,Label
0,ID_000012eaf_any,0.001241
1,ID_000012eaf_epidural,0.000114
2,ID_000012eaf_intraparenchymal,0.000130
3,ID_000012eaf_intraventricular,0.000024
4,ID_000012eaf_subarachnoid,0.000149
5,ID_000012eaf_subdural,0.000968
6,ID_0000ca2f6_any,0.001707
7,ID_0000ca2f6_epidural,0.000060
8,ID_0000ca2f6_intraparenchymal,0.000250
9,ID_0000ca2f6_intraventricular,0.000027


(471270, 2)

In [62]:
#!/home/reina/anaconda3/bin/kaggle competitions submit rsna-intracranial-hemorrhage-detection -f /media/hd/notebooks/data/RSNA/submissions/submission57.csv -m "the 5 folds models including focal, mean after sigmoid"

100%|███████████████████████████████████████| 16.8M/16.8M [02:10<00:00, 135kB/s]
Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [63]:
sub=pd.read_csv('/media/hd/notebooks/data/RSNA/submissions/submission55.csv')

In [64]:
sub0=pd.read_csv('/media/hd/notebooks/data/RSNA/submissions/submission54.csv')

In [66]:
F.binary_cross_entropy(torch.tensor(sub.Label.values),torch.tensor(sub0.Label.values))

tensor(0.0487, dtype=torch.float64)

In [70]:
F.binary_cross_entropy(torch.tensor(sub.Label.values,dtype=torch.float),torch.tensor(submission_df.Label.values,dtype=torch.float))

tensor(0.1125)

In [78]:
F.binary_cross_entropy(torch.tensor(sub.Label.values,dtype=torch.float),torch.tensor(submission_df.Label.values,dtype=torch.float))

tensor(0.0484)

In [112]:
F.binary_cross_entropy(torch.tensor(sub.Label.values,dtype=torch.float),torch.tensor(submission_df.Label.values,dtype=torch.float))

tensor(0.0486)